In [7]:
from warnings import filterwarnings
filterwarnings("ignore")
%config IPCompleter.use_jedi=False
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [2]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("income_evaluation.csv")
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [10]:
df.isnull().sum()

age                0
 workclass         0
 fnlwgt            0
 education         0
 education-num     0
 marital-status    0
 occupation        0
 relationship      0
 race              0
 sex               0
 capital-gain      0
 capital-loss      0
 hours-per-week    0
 native-country    0
 income            0
dtype: int64

In [22]:
df.columns

Index(['age', ' workclass', ' fnlwgt', ' education', ' education-num',
       ' marital-status', ' occupation', ' relationship', ' race', ' sex',
       ' capital-gain', ' capital-loss', ' hours-per-week', ' native-country',
       ' income'],
      dtype='object')

In [23]:
X=df.drop(" income",axis=1)
Y=df[" income"]
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.23,random_state=42)

In [13]:
numeric_columns=X.select_dtypes(include="number").columns
categorical_columns=X.select_dtypes(include="object").columns

In [15]:
numerical_Pipeline = Pipeline(
    steps=[
    ('scaler', StandardScaler())
])

categorical_Pipeline = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

In [17]:
preprocessor = ColumnTransformer(
    transformers=[
        ('categorical', categorical_Pipeline, categorical_columns),
        ('numerical', numerical_Pipeline, numeric_columns)
    ])

In [18]:
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

In [19]:
pipe.fit(X_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index([' workclass', ' education', ' marital-status', ' occupation',
       ' relationship', ' race', ' sex', ' native-country'],
      dtype='object')),
                                                 ('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['age', ' fnlwgt', ' education-num', ' capital-gain', ' capital-loss',
       ' hours-per-week'],
      dtype='object'))]))])

In [25]:
X_train = pipe.transform(X_train)
X_test = pipe.transform(X_test)

In [27]:
X_train

<25071x108 sparse matrix of type '<class 'numpy.float64'>'
	with 350994 stored elements in Compressed Sparse Row format>

In [28]:
import xgboost as xgb
model=xgb.XGBClassifier(random_state=1,learning_rate=0.01)
model=model.fit(X_train,y_train)
y_pred=model.predict(X_test)
y_pred

[16:55:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


array([' <=50K', ' <=50K', ' >50K', ..., ' <=50K', ' <=50K', ' <=50K'],
      dtype=object)

In [29]:
from sklearn.metrics import accuracy_score,confusion_matrix
print("Confusion Matrix \n",confusion_matrix(y_test,y_pred))
accuracy_score(y_test,y_pred)

Confusion Matrix 
 [[5460  250]
 [ 820  960]]


0.8571428571428571

In [30]:
import pickle
document="PipeTask1"
pickle.dump(model,open(document,"wb"))

In [32]:
loaded_model=pickle.load(open(document,'rb'))
y_loded_model_pred=loaded_model.predict(X_test)

In [33]:
y_loded_model_pred

array([' <=50K', ' <=50K', ' >50K', ..., ' <=50K', ' <=50K', ' <=50K'],
      dtype=object)